# RNN Predict Observation at Next Time Step

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

## Prepare Input Data

In [2]:
npzfile = np.load("data/train.npz", allow_pickle=True)
xraw_train = npzfile['x']
y_train = npzfile['y']
s = 4 # alphabet size

In [3]:
# convert variable length lists to fixed-length array by 
# clipping longer sequences and padding shorter sequences with -1
x_train = tf.keras.preprocessing.sequence.pad_sequences(
    xraw_train, padding="post", maxlen=99, value = -1)
# Add 1 so 0 stands for missing and 1,2,3,4 for a,c,g,t
x_train = x_train + 1

In [4]:
x_train[0]

array([2, 4, 4, 4, 3, 3, 3, 1, 3, 3, 2, 2, 1, 1, 3, 1, 4, 3, 3, 3, 4, 3,
       3, 1, 4, 2, 1, 2, 4, 4, 3, 1, 3, 3, 4, 2, 1, 3, 3, 1, 3, 4, 4, 2,
       3, 1, 3, 1, 2, 2, 1, 3, 2, 2, 4, 3, 3, 2, 2, 1, 1, 2, 1, 4, 3, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [5]:
# x_train = tf.one_hot(x_train, s)

In [6]:
print ("input shape: ", x_train.shape)
x_train[:2,:]

input shape:  (12138, 99)


array([[2, 4, 4, 4, 3, 3, 3, 1, 3, 3, 2, 2, 1, 1, 3, 1, 4, 3, 3, 3, 4, 3,
        3, 1, 4, 2, 1, 2, 4, 4, 3, 1, 3, 3, 4, 2, 1, 3, 3, 1, 3, 4, 4, 2,
        3, 1, 3, 1, 2, 2, 1, 3, 2, 2, 4, 3, 3, 2, 2, 1, 1, 2, 1, 4, 3, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [3, 3, 2, 2, 2, 4, 3, 3, 3, 2, 1, 3, 2, 2, 3, 4, 4, 3, 1, 1, 2, 4,
        4, 4, 2, 2, 2, 4, 2, 4, 2, 1, 4, 2, 1, 3, 2, 2, 2, 4, 3, 2, 2, 1,
        2, 2, 1, 3, 3, 2, 1, 3, 3, 1, 2, 4, 2, 1, 3, 1, 3, 3, 4, 2, 2, 2,
        2, 4, 4, 2, 2, 4, 2, 4, 3, 4, 2, 2, 2, 2, 4, 3, 2, 4, 3, 2, 1, 3,
        3, 4, 3, 3, 1, 1, 3, 1, 4, 3, 1]], dtype=int32)

## Make TF Dataset
Here, we do not need the labels (exon or not). Instead we try to predict the $i$-th character from input sequence up to $i-1$. E.g. the training output 
from "attcac" could be "ttcacg".

In [7]:
def future_sequence(sequence):
    input_seq = sequence[:-1]
    target_seq = sequence[1:]
    target_seq = tf.one_hot(target_seq, depth=5)
    target_seq = target_seq[:,1:]
    return input_seq, target_seq

future_sequence([1,4,4,1,1,2,3])

([1, 4, 4, 1, 1, 2],
 <tf.Tensor: shape=(6, 4), dtype=float32, numpy=
 array([[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]], dtype=float32)>)

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices(x_train)
train_ds = train_ds.map(future_sequence)
train_ds = train_ds.shuffle(20000).batch(32, drop_remainder=True)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

for (a, b) in train_ds.take(1):
    print("shape of training example", a.shape, "output", b.shape)
    print("training example", a[0,:10], "output", b[0,:10])

shape of training example (32, 98) output (32, 98, 4)
training example tf.Tensor([2 4 4 2 4 4 1 3 2 2], shape=(10,), dtype=int32) output tf.Tensor(
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]], shape=(10, 4), dtype=float32)


## Prepare Model

In [13]:
units=64
rnn_simple = tf.keras.Sequential()
rnn_simple.add(layers.Embedding(input_dim=s+1, output_dim=4, mask_zero=True))
rnn_simple.add(layers.GRU(units, return_sequences=True))
# SimpleRNN outputs the hidden states directly.
# Here, we want to transform them with a Dense layer with a units x s parameter matrix.
# The following tf.keras.layers.Dense layer is applied to all positions and produces 
# a sequence output.
rnn_simple.add(layers.Dense(4, activation='softmax'))
rnn_simple.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 4)           20        
_________________________________________________________________
gru (GRU)                    (None, None, 64)          13440     
_________________________________________________________________
dense_1 (Dense)              (None, None, 4)           260       
Total params: 13,720
Trainable params: 13,720
Non-trainable params: 0
_________________________________________________________________


In [14]:
out = rnn_simple(x_train[0:10,:])
out.shape

TensorShape([10, 99, 4])

# Training

In [15]:
rnn_simple.compile(optimizer='adam', loss='categorical_crossentropy')

In [16]:
history = rnn_simple.fit(train_ds, epochs=20)

Epoch 1/20
379/379 [==============================] - 12s 26ms/step - loss: 1.2188
Epoch 2/20
379/379 [==============================] - 10s 26ms/step - loss: 1.1903
Epoch 3/20
379/379 [==============================] - 11s 27ms/step - loss: 1.1916
Epoch 4/20
379/379 [==============================] - 11s 27ms/step - loss: 1.1887
Epoch 5/20
379/379 [==============================] - 11s 29ms/step - loss: 1.1878
Epoch 6/20
379/379 [==============================] - 11s 28ms/step - loss: 1.1858
Epoch 7/20
379/379 [==============================] - 11s 28ms/step - loss: 1.1888
Epoch 8/20
379/379 [==============================] - 11s 28ms/step - loss: 1.1877
Epoch 9/20
379/379 [==============================] - 11s 28ms/step - loss: 1.1846
Epoch 10/20
379/379 [==============================] - 11s 29ms/step - loss: 1.1857
Epoch 11/20
379/379 [==============================] - 11s 28ms/step - loss: 1.1865
Epoch 12/20
379/379 [==============================] - 11s 28ms/step - loss: 1.1856
E

In [17]:
np.log(4)

1.3862943611198906